In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
df=pd.read_csv('/content/movies.csv')

In [8]:
df

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,4798,220000,Action Crime Thriller,NaN,9367,united states\u2013mexico barrier legs arms pa...,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,...,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,"[{'name': 'Robert Rodriguez', 'gender': 0, 'de...",Robert Rodriguez
4799,4799,9000,Comedy Romance,NaN,72766,NaN,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,...,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,"[{'name': 'Edward Burns', 'gender': 2, 'depart...",Edward Burns
4800,4800,0,Comedy Drama Romance TV Movie,http://www.hallmarkchannel.com/signedsealeddel...,231617,date love at first sight narration investigati...,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,...,120.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,NaN,"Signed, Sealed, Delivered",7.0,6,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,"[{'name': 'Carla Hetland', 'gender': 0, 'depar...",Scott Smith
4801,4801,0,NaN,http://shanghaicalling.com/,126186,NaN,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,...,98.0,"[{""iso_639_1"": ""en

In [9]:
df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


Feature Selecting Phase

In [12]:
features =['genres','keywords','tagline','cast','director','cast','title']
print(features)

['genres', 'keywords', 'tagline', 'cast', 'director', 'cast', 'title']


In [13]:
for feature in features:
  df[feature]=df[feature].fillna('')

In [14]:
combine_features = df['genres']+' '+df['keywords']+' '+df['tagline']+' '+df['cast']+' '+df['director']+' '+df['title']

In [15]:
print(combine_features)

0       Action Adventure Fantasy Science Fiction cultu...
1       Adventure Fantasy Action ocean drug abuse exot...
2       Action Adventure Crime spy based on novel secr...
3       Action Crime Drama Thriller dc comics crime fi...
4       Action Adventure Science Fiction based on nove...
                              ...                        
4798    Action Crime Thriller united states\u2013mexic...
4799    Comedy Romance  A newlywed couple's honeymoon ...
4800    Comedy Drama Romance TV Movie date love at fir...
4801      A New Yorker in Shanghai Daniel Henney Eliza...
4802    Documentary obsession camcorder crush dream gi...
Length: 4803, dtype: object


Converting text to numerical values

In [16]:
vectorizer = TfidfVectorizer()

In [17]:
feature_vectors = vectorizer.fit_transform(combine_features)
print(feature_vectors)

  (0, 1183)	0.2771429775697421
  (0, 2720)	0.16779665077750835
  (0, 8651)	0.10950771233518673
  (0, 14400)	0.18869504935935053
  (0, 11288)	0.15600564366704217
  (0, 9709)	0.22061174669983702
  (0, 16170)	0.1471845509560594
  (0, 18477)	0.19277176743945276
  (0, 15542)	0.2000852661458036
  (0, 14718)	0.21153518149440184
  (0, 19148)	0.19502634639381392
  (0, 18845)	0.22968831190527222
  (0, 14749)	0.14568185359096344
  (0, 12700)	0.2552737122112953
  (0, 12356)	0.0757123618230531
  (0, 18835)	0.12179929157015998
  (0, 16904)	0.05365726945306951
  (0, 5519)	0.22061174669983702
  (0, 15785)	0.2044948505609709
  (0, 3587)	0.24248101205828376
  (0, 18385)	0.12067236175146148
  (0, 15901)	0.3255851082321633
  (0, 6519)	0.15884357175977004
  (0, 3408)	0.21574818782392274
  (0, 4103)	0.20781904654682548
  :	:
  (4801, 7731)	0.25682086501772416
  (4801, 12869)	0.1918304797254379
  (4801, 1880)	0.1383742924803071
  (4801, 12062)	0.1190062927991386
  (4801, 8323)	0.094776382167548
  (4801, 4234

Getting similarity scores using cosine similarity

In [18]:
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.07294698 0.03533251 ... 0.         0.         0.        ]
 [0.07294698 1.         0.02792771 ... 0.04419983 0.         0.        ]
 [0.03533251 0.02792771 1.         ... 0.         0.04636139 0.        ]
 ...
 [0.         0.04419983 0.         ... 1.         0.         0.05551043]
 [0.         0.         0.04636139 ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.05551043 0.         1.        ]]


In [19]:
movie_name = input('Enter the movie name: ')

Enter the movie name: fast and furious


In [20]:
list_of_movies = df['title'].tolist()
print(list_of_movies)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [22]:
find_close_match = difflib.get_close_matches(movie_name,list_of_movies)
print(find_close_match)

['The Fast and the Furious', '2 Fast 2 Furious']


In [23]:
close_match = find_close_match[0]
print(close_match)

The Fast and the Furious


Find the index of the movie

In [24]:
index_of_movie = df[df.title == close_match]['index'].values[0]
print(index_of_movie)

99


Getting similiar movies

In [25]:
similarity_score = list(enumerate(similarity[index_of_movie]))
print(similarity_score)

[(0, 0.06742353162078606), (1, 0.06760131928009126), (2, 0.013587560407619614), (3, 0.03928989262693607), (4, 0.005464619637040736), (5, 0.025239470923048218), (6, 0.0), (7, 0.0050941631878943195), (8, 0.01433119084603225), (9, 0.004962509372090892), (10, 0.011807924216843324), (11, 0.04224878830558148), (12, 0.010238728284860128), (13, 0.014209866189613327), (14, 0.010698860564794003), (15, 0.004828337316636382), (16, 0.010406929016923383), (17, 0.058890924590120894), (18, 0.06084653519851771), (19, 0.02661324550456186), (20, 0.015869521958732414), (21, 0.019913334964301345), (22, 0.012730602657927192), (23, 0.012601522846713719), (24, 0.015425793604769578), (25, 0.0048818405539415854), (26, 0.005168424807773607), (27, 0.03498612454948252), (28, 0.016012092744225583), (29, 0.011001413038734578), (30, 0.019199252574600548), (31, 0.0161480572092454), (32, 0.0), (33, 0.015414841941687577), (34, 0.0), (35, 0.010652306418233542), (36, 0.004539334243916116), (37, 0.0363797805110102), (38, 0

Sorting the list on similarity score

In [26]:
sorted_movies = sorted(similarity_score,key=lambda x:x[1],reverse=True)
print(sorted_movies)

[(99, 0.9999999999999996), (44, 0.3602180949935334), (500, 0.3526421038800094), (204, 0.3212305975857114), (405, 0.23973131344177404), (658, 0.1727307141124062), (568, 0.16830764699447195), (2548, 0.1616885900655865), (3697, 0.15178304029186002), (566, 0.14734586420709064), (574, 0.145861142948663), (2541, 0.1419491035497093), (3297, 0.1347078997817714), (1186, 0.1312847673060466), (2154, 0.12546484434092722), (1882, 0.1227329488097651), (3142, 0.12160176121354845), (2546, 0.11725175073686227), (1083, 0.11708232513228213), (1141, 0.11655469679085474), (1399, 0.11182154147184366), (223, 0.110742413517472), (3848, 0.10746763264380171), (1395, 0.10551501688281253), (4623, 0.10505393021556858), (807, 0.10491243498109966), (2775, 0.10449377397640482), (4467, 0.10190562526268825), (3420, 0.10072240382196732), (2682, 0.09962054289779426), (963, 0.0994334399868273), (3012, 0.09811502844369356), (1350, 0.09736581553547373), (1995, 0.09677577832529992), (3173, 0.09618215685075827), (2225, 0.0955

In [27]:
print('Suggestions: \n')
i=1
for movie in sorted_movies:
  index=movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if(i<16):
    print(i,'.',title_from_index)
    i+=1

Suggestions: 

1 . The Fast and the Furious
2 . Furious 7
3 . 2 Fast 2 Furious
4 . Fast Five
5 . The Fast and the Furious: Tokyo Drift
6 . Death Race
7 . xXx
8 . Malibu's Most Wanted
9 . Fast Times at Ridgemont High
10 . Cars
11 . S.W.A.T.
12 . The Faculty
13 . The Death and Life of Bobby Z
14 . The Final Destination
15 . Street Kings


Final interface

In [28]:
movie_name = input('Enter the movie name: ')

list_of_movies = df['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name,list_of_movies)

close_match = find_close_match[0]

index_of_movie = df[df.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_movie]))

sorted_movies = sorted(similarity_score,key=lambda x:x[1],reverse=True)

print('Suggestions: \n')
i=1
for movie in sorted_movies:
  index=movie[0]
  title_from_index = df[df.index==index]['title'].values[0]
  if(i<16):
    print(i,'.',title_from_index)
    i+=1

Enter the movie name: the terminal
Suggestions: 

1 . The Terminal
2 . Roll Bounce
3 . Saving Private Ryan
4 . The Mask of Zorro
5 . The Interpreter
6 . The Bad News Bears
7 . Ocean's Twelve
8 . Side Effects
9 . Larry Crowne
10 . The Book of Life
11 . Home Alone 2: Lost in New York
12 . No Reservations
13 . Entrapment
14 . In Too Deep
15 . Mickey Blue Eyes
